# Crawl KnoWhys talks from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

In [ ]:
# config
host = 'https://knowhy.bookofmormoncentral.org/reference-knowhy'
base_dir = '../data/raw/knowhys'
bs_parser = 'html.parser'
delay_seconds = 5

In [ ]:
def extract_hrefs_from_views_field_title(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    divs_with_class = soup.find_all('div', class_='views-field-title')

    for div in divs_with_class:
        # Find all anchor tags within the div
        anchor_tags = div.find_all('a')

        # Extract href attribute from each anchor tag and append to the list
        for anchor in anchor_tags:
            href = anchor.get('href')
            if href:
                hrefs.append(urljoin(host, href))

    return hrefs

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    return os.path.join(base_dir, f"{path_components[-1]}.json")
    

In [ ]:
knowhy_links = []

page_counter = 0
while True:
    print(page_counter)
    status_code, html = get_page("https://knowhy.bookofmormoncentral.org/reference-knowhy?page="+str(page_counter), delay_seconds)
    if status_code != 200:
        print("Error!", status_code, page_counter)
        continue
    hrefs = extract_hrefs_from_views_field_title(html)
    if len(hrefs) == 0:
        break
    else:
        knowhy_links.extend(hrefs)
        page_counter += 1

In [ ]:
for url in knowhy_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")